In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun  9 08:17:57 2023

@author: Joao

This script reads a training CSV, outputs predictions for the data, and applies
the score metric to compute the final score. 

It also computes a few other known metrics, like top-k accuracy and average power loss.


"""

# 'F5' Começa a debuger o codigo 
# 'F10' Analisar a linha sem entrar no codigo 
# 'F11' Analisar linha  e entrar no codigo 
# 'SHIFT-F11' sair do bloco de codigo atual e continuar a execução


import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from tqdm import tqdm

X_SIZE = 5      # 5 input samples
N_GPS = 2       # 2 GPSs (unit1 and unit2)
N_GPS_COORD = 2 # 2 GPS coords (latitude & longitude)
N_ARR = 4       # 4 arrays
N_BEAMS = 64    # 64 beams per array
IDX_COL1 = 'unique_index' # index col in the training CSV
IDX_COL2 = 'abs_index'    # index col in the CSVs of the V2V dataset 
                          # this indices are the same, just dif names

def norm_2pi(x):
    x_normed = np.empty_like(x)
    x_normed[:] = x
    for i in range(len(x)):
        if abs(x[i]) >= np.pi:
            x_normed[i] = x[i] % (2*np.pi)

            if x[i] >= np.pi:
                x_normed[i] -= 2*np.pi

        while x_normed[i] < -np.pi:
            x_normed[i] += 2*np.pi

        while x_normed[i] > np.pi:
            x_normed[i] -= 2*np.pi

        if x_normed[i] < -np.pi:
            print(f'{i}, {x_normed[i]}')

    return x_normed


def compute_ori_from_pos_delta(lat_deltas, lon_deltas):

    n_samples = len(lat_deltas)
    pose = np.zeros(n_samples)

    for i in range(n_samples):

        delta_lat = lat_deltas[i-1]
        delta_lon = lon_deltas[i-1]

        if delta_lon == 0:
            if delta_lat == 0:
                pose[i] = 0
                continue
            elif delta_lat > 0:
                slope = np.pi / 2
            elif delta_lat < 0:
                slope = -np.pi / 2
        else:
            slope = np.arctan(delta_lat / delta_lon)
            if delta_lat == 0:
                slope = np.pi if delta_lon < 0 else 0
            elif delta_lat < 0 and delta_lon < 0:
                slope = -np.pi + slope
            elif delta_lon < 0 and delta_lat > 0:
                slope = np.pi + slope

        pose[i] = slope

    return pose


def estimate_positions(input_positions, delta_input, delta_output):

        # Calculate the number of samples in the input positions array
    n_samples = input_positions.shape[0]

    # Initialize an array to store the estimated output positions
    out_pos = np.zeros((n_samples, 2))

    # Determine the size of each input position array (number of samples)
    x_size = input_positions.shape[1]

    # Define the time points corresponding to each sample in the input positions array
    x = delta_input * np.arange(x_size)

    # Iterate over each sample to estimate the corresponding output position
    for sample_idx in tqdm(range(n_samples), desc='Estimating input positions'):
        # Extract the input positions for the current sample
        input_pos = input_positions[sample_idx]

        # Perform linear interpolation to estimate latitude and longitude at the output time
        f_lat = scipy.interpolate.interp1d(x, input_pos[:, 0], fill_value='extrapolate')
        f_lon = scipy.interpolate.interp1d(x, input_pos[:, 1], fill_value='extrapolate')

        # Calculate the estimated latitude and longitude at the output time
        out_pos[sample_idx, 0] = f_lat(x[-1] + delta_output)
        out_pos[sample_idx, 1] = f_lon(x[-1] + delta_output)

    # Return the array of estimated output positions
    return out_pos



def predict_beam_uniformly_from_aoa(aoa):

    beam_predictions = np.zeros_like(aoa)

    beam_ori = np.arange(N_BEAMS * N_ARR) / (N_BEAMS * N_ARR - 1) * 2*np.pi - np.pi

    angl_diff_to_each_beam = aoa.reshape((-1, 1)) - beam_ori

    beam_predictions = np.argsort(abs(angl_diff_to_each_beam), axis=1)

    return beam_predictions


def circular_distance(a, b, l=256, sign=False):

    while a < 0:
        a = l - abs(a)
    while b < 0:
        b = l - abs(b)
        
    a = a % l if a >= l else a
    b = b % l if b >= l else b
    
    dist = a - b

    if abs(dist) > l/2:
        dist = l - abs(dist)

    return dist if sign else abs(dist)


def compute_acc(all_beams, only_best_beam, top_k=[1, 3, 5]):
    

    n_top_k = len(top_k)
    total_hits = np.zeros(n_top_k)

    n_test_samples = len(only_best_beam)
    if len(all_beams) != n_test_samples:
        raise Exception(
            'Number of predicted beams does not match number of labels.')

    # For each test sample, count times where true beam is in k top guesses
    for samp_idx in range(len(only_best_beam)):
        for k_idx in range(n_top_k):
            hit = np.any(all_beams[samp_idx, :top_k[k_idx]] == only_best_beam[samp_idx])
            total_hits[k_idx] += 1 if hit else 0

    # Average the number of correct guesses (over the total samples)
    return np.round(total_hits / len(only_best_beam), 4)


def APL(true_best_pwr, est_best_pwr):

    
    return np.mean(10 * np.log10(est_best_pwr / true_best_pwr))


In [3]:
import logging
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os 
import torch
from tqdm import tqdm
logging.basicConfig(level=logging.INFO)

core_path = r'C:\Python\BeamPrediction_DeepLearning\BeamPrediction_DeepLearning\data'

scen_idx = 36
training_path ='deepsense_challenge2023_trainset.csv'
testing_path = 'deepsense_challenge2023_testset_example'
csv_train = os.path.join(core_path, training_path)
csv_dict_path = os.path.join(core_path, 'scenario36.p')

X_SIZE = 5
N_GPS = 2
N_GPS_COORD = 2
N_ARR = 4
N_BEAMS = 64
IDX_COL1 = 'unique_index'
IDX_COL2 = 'abs_index'

# Carregar dados
with open(csv_dict_path, 'rb') as fp:
    csv_dict = pickle.load(fp)

df_train = pd.read_csv(csv_train)

if True:  # TREINAMENTO
    # try:
        # Preparar os dados
        samples_of_scen = np.where(df_train['scenario'] == scen_idx)[0][1000:23187]

In [4]:
samples_of_scen.shape

(22187,)

In [2]:
import torch

# Verificar se CUDA está disponível
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA está disponível. Utilizando GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA não está disponível. Utilizando CPU.")


CUDA está disponível. Utilizando GPU: NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [3]:
import torch
print(torch.__version__)
print(torch.version.cuda)


2.4.0+cu118
11.8


In [4]:

scen_idx = 36
csv_train = r'C:\Python\BeamPrediction_DeepLearning\BeamPrediction_DeepLearning\data\deepsense_challenge2023_trainset.csv'
csv_dict_path = rf'C:\Python\BeamPrediction_DeepLearning\BeamPrediction_DeepLearning\data\scenario36.p'

with open(csv_dict_path, 'rb') as fp:
    csv_dict = pickle.load(fp)

df_train = pd.read_csv(csv_train)
df_train.head()


,scenario,x1_unique_index,x1_unit1_gps1,x1_unit2_gps1,x1_unit1_rgb5,x1_unit1_rgb6,x2_unique_index,x2_unit1_gps1,x2_unit2_gps1,x2_unit1_rgb5,...,x5_unit1_gps1,x5_unit2_gps1,x5_unit1_rgb5,x5_unit1_rgb6,y1_unique_index,y1_unit1_overall-beam,y1_unit1_pwr1,y1_unit1_pwr2,y1_unit1_pwr3,y1_unit1_pwr4
0,36,2674,scenario36/unit1/gps1/gps_8869_11-46-31.233334...,scenario36/unit2/gps1/gps_33301_11-46-31.25000...,scenario36/unit1/rgb5/frame_11-46-31.205818.jpg,scenario36/unit1/rgb6/frame_11-46-31.205818.jpg,2676,scenario36/unit1/gps1/gps_8871_11-46-31.400000...,scenario36/unit2/gps1/gps_33303_11-46-31.41666...,scenario36/unit1/rgb5/frame_11-46-31.406020.jpg,...,scenario36/unit1/gps1/gps_8877_11-46-32.000000...,scenario36/unit2/gps1/gps_33310_11-46-32.00000...,scenario36/unit1/rgb5/frame_11-46-32.006626.jpg,scenario36/unit1/rgb6/frame_11-46-32.006626.jpg,2687,161,scenario36/unit1/pwr1/pwr_11-46-31.859441.txt,scenario36/unit1/pwr2/pwr_11-46-31.859441.txt,scenario36/unit1/pwr3/pwr_11-46-31.859441.txt,scenario36/unit1/pwr4/pwr_11-46-31.859441.txt
1,36,2675,scenario36/unit1/gps1/gps_8870_11-46-31.300000...,scenario36/unit2/gps1/gps_33302_11-46-31.33333...,scenario36/unit1/rgb5/frame_11-46-31.305919.jpg,scenario36/unit1/rgb6/frame_11-46-31.305919.jpg,2677,scenario36/unit1/gps1/gps_8872_11-46-31.500000...,scenario36/unit2/gps1/gps_33304_11-46-31.50000...,scenario36/unit1/rgb5/frame_11-46-31.506121.jpg,...,scenario36/unit1/gps1/gps_8879_11-46-32.133334...,scenario36/unit2/gps1/gps_33311_11-46-32.08333...,scenario36/unit1/rgb5/frame_11-46-32.106727.jpg,scenario36/unit1/rgb6/frame_11-46-32.106727.jpg,2688,161,scenario36/unit1/pwr1/pwr_11-46-31.959645.txt,scenario36/unit1/pwr2/pwr_11-46-31.959645.txt,scenario36/unit1/pwr3/pwr_11-46-31.959645.txt,scenario36/unit1/pwr4/pwr_11-46-31.959645.txt
2,36,2676,scenario36/unit1/gps1/gps_8871_11-46-31.400000...,scenario36/unit2/gps1/gps_33303_11-46-31.41666...,scenario36/unit1/rgb5/frame_11-46-31.406020.jpg,scenario36/unit1/rgb6/frame_11-46-31.406020.jpg,2678,scenario36/unit1/gps1/gps_8873_11-46-31.600000...,scenario36/unit2/gps1/gps_33305_11-46-31.58333...,scenario36/unit1/rgb5/frame_11-46-31.606222.jpg,...,scenario36/unit1/gps1/gps_8880_11-46-32.200000...,scenario36/unit2/gps1/gps_33313_11-46-32.25000...,scenario36/unit1/rgb5/frame_11-46-32.206828.jpg,scenario36/unit1/rgb6/frame_11-46-32.206828.jpg,2689,161,scenario36/unit1/pwr1/pwr_11-46-32.059402.txt,scenario36/unit1/pwr2/pwr_11-46-32.059402.txt,scenario36/unit1/pwr3/pwr_11-46-32.059402.txt,scenario36/unit1/pwr4/pwr_11-46-32.059402.txt
3,36,2677,scenario36/unit1/gps1/gps_8872_11-46-31.500000...,scenario36/unit2/gps1/gps_33304_11-46-31.50000...,scenario36/unit1/rgb5/frame_11-46-31.506121.jpg,scenario36/unit1/rgb6/frame_11-46-31.506121.jpg,2679,scenario36/unit1/gps1/gps_8874_11-46-31.700000...,scenario36/unit2/gps1/gps_33307_11-46-31.75000...,scenario36/unit1/rgb5/frame_11-46-31.706323.jpg,...,scenario36/unit1/gps1/gps_8881_11-46-32.300000...,scenario36/unit2/gps1/gps_33314_11-46-32.33333...,scenario36/unit1/rgb5/frame_11-46-32.306929.jpg,scenario36/unit1/rgb6/frame_11-46-32.306929.jpg,2690,161,scenario36/unit1/pwr1/pwr_11-46-32.160005.txt,scenario36/unit1/pwr2/pwr_11-46-32.160005.txt,scenario36/unit1/pwr3/pwr_11-46-32.160005.txt,scenario36/unit1/pwr4/pwr_11-46-32.160005.txt
4,36,2678,scenario36/unit1/gps1/gps_8873_11-46-31.600000...,scenario36/unit2/gps1/gps_33305_11-46-31.58333...,scenario36/unit1/rgb5/frame_11-46-31.606222.jpg,scenario36/unit1/rgb6/frame_11-46-31.606222.jpg,2680,scenario36/unit1/gps1/gps_8875_11-46-31.800000...,scenario36/unit2/gps1/gps_33308_11-46-31.83333...,scenario36/unit1/rgb5/frame_11-46-31.806424.jpg,...,scenario36/unit1/gps1/gps_8882_11-46-32.400000...,scenario36/unit2/gps1/gps_33315_11-46-32.41666...,scenario36/unit1/rgb5/frame_11-46-32.407030.jpg,scenario36/unit1/rgb6/frame_11-46-32.407030.jpg,2691,161,scenario36/unit1/pwr1/pwr_11-46-32.260057.txt,scenario36/unit1/pwr2/pwr_11-46-32.260057.txt,scenario36/unit1/pwr3/pwr_11-46-32.260

In [5]:

# Load all positions
samples_of_scen = np.where(df_train['scenario'] == scen_idx)[0]
n_samples = len(samples_of_scen)

X_SIZE = 5      # 5 input samples
N_GPS = 2       # 2 GPSs (unit1 and unit2)
N_GPS_COORD = 2 # 2 GPS coords (latitude & longitude)
N_ARR = 4       # 4 arrays
N_BEAMS = 64    # 64 beams per array
IDX_COL1 = 'unique_index' # index col in the training CSV
IDX_COL2 = 'abs_index'    # index col in the CSVs of the V2V dataset 
                          # this indices are the same, just dif names
loaded_positions = set()
train_positions = np.zeros((n_samples, X_SIZE, N_GPS, N_GPS_COORD))
y_pos1 = np.zeros((n_samples, N_GPS_COORD))
y_pos2 = np.zeros((n_samples, N_GPS_COORD))
y_pwrs = np.zeros((n_samples, N_ARR, N_BEAMS))

# Loop over each sample in the training data
for sample_idx in tqdm(range(n_samples), desc='Loading data'):
    # Get the current training sample
    train_sample = samples_of_scen[sample_idx]
    
    # Loop over each position index on the X axis
    for x_idx in range(X_SIZE):
        # Find the absolute relative index in the CSV DataFrame for the current training sample
        abs_idx_relative_index = (csv_dict[IDX_COL2] == df_train[f'x{x_idx+1}_'+IDX_COL1][train_sample])
        
        # Fill the training positions for the current sample and current X index
        # GPS positions of 'unit1' and 'unit2' are stored in train_positions
        # Index [0, :] is used for 'unit1' GPS coordinates, and index [1, :] is used for 'unit2' GPS coordinates
        train_positions[sample_idx, x_idx, 0, :] = csv_dict['unit1_gps1'][abs_idx_relative_index]
        train_positions[sample_idx, x_idx, 1, :] = csv_dict['unit2_gps1'][abs_idx_relative_index]

    # Find the index for the ground truth position 'y' in the CSV dictionary DataFrame
    y_idx = (csv_dict[IDX_COL2] == df_train['y1_'+IDX_COL1][train_sample])

    # Store the GPS positions for 'unit1' and 'unit2' corresponding to the ground truth 'y' position
    y_pos1[sample_idx] = csv_dict['unit1_gps1'][y_idx]
    y_pos2[sample_idx] = csv_dict['unit2_gps1'][y_idx]

    # Loop over each antenna array to store the power readings corresponding to the ground truth 'y' position
    for arr_idx in range(N_ARR):
        y_pwrs[sample_idx, arr_idx] = csv_dict[f'unit1_pwr{arr_idx+1}'][y_idx]


Loading data: 100%|██████████| 23187/23187 [00:26<00:00, 886.63it/s] 


In [19]:
train_positions.shape

(23187, 5, 2, 2)

In [20]:
y_pos2.shape , y_pos1.shape , y_pwrs.shape

((23187, 2), (23187, 2), (23187, 4, 64))

In [21]:
# Extract ground truth beams for 'unit1' from the DataFrame, based on the selected samples
y_true_beams = df_train['y1_unit1_overall-beam'].values[samples_of_scen] 

y_pwrs_reshaped = y_pwrs.reshape((n_samples, -1)) # (23187, 256)
# Sort the power readings in descending order to get the indices of the beams with highest power
all_true_beams = np.flip(np.argsort(y_pwrs_reshaped, axis=1), axis=1) # (23187, 256)


In [23]:
all_true_beams.shape, y_pwrs_reshaped.shape

((23187, 256), (23187, 256))

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

class CNN(nn.Module):
    def __init__(self, input_size):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.batch_norm1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.batch_norm2 = nn.BatchNorm1d(128)
        self.fc1 = nn.Linear(128 * input_size, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)  # Saída para as posições latitudinal e longitudinal

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.batch_norm1(x)
        x = torch.relu(self.conv2(x))
        x = self.batch_norm2(x)
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def train_cnn(input_positions, delta_input, delta_output, epochs=7):
    n_samples = input_positions.shape[0]
    x_size = input_positions.shape[1]
    
    # Preparando os dados de entrada e saída
    X_train = torch.zeros((n_samples, 1, x_size)).to(device)
    y_train = torch.zeros((n_samples, 2)).to(device)
    
    for sample_idx in range(n_samples):
        input_pos = input_positions[sample_idx]
        X_train[sample_idx, 0, :] = torch.tensor(input_pos[:, 0], dtype=torch.float32)  # Usando apenas a primeira coluna como entrada
        y_train[sample_idx, 0] = input_pos[-1, 0] + delta_output  # Prevendo a posição latitudinal
        y_train[sample_idx, 1] = input_pos[-1, 1] + delta_output  # Prevendo a posição longitudinal
    
    model = CNN(x_size).to(device)
    criterion = nn.SmoothL1Loss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    
    # Treinando o modelo
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for i in range(n_samples):
            inputs, labels = X_train[i:i+1], y_train[i:i+1]
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {running_loss / n_samples}')
    
    return model
def estimate_positions(input_positions, delta_input, delta_output, model, device='cuda'):
    n_samples = input_positions.shape[0]
    out_pos = np.zeros((n_samples, 2))

    x_size = input_positions.shape[1]
    time_steps = delta_input * np.arange(x_size)  # Vetor de tempo para as entradas
    model.eval()  # Coloca o modelo em modo de avaliação (desativa o dropout e outras camadas específicas de treino)

    # Itera sobre cada amostra para estimar a posição correspondente
    for sample_idx in tqdm(range(n_samples), desc='Estimating input positions'):
        input_pos = input_positions[sample_idx]

        # Ajusta o tempo final, considerando o delta_output
        final_time = time_steps[-1] + delta_output

        # Converte a entrada para um tensor PyTorch e move para o dispositivo especificado (CPU ou GPU)
        inputs = torch.tensor(input_pos[:, 0], dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
        
        # Gera a previsão utilizando o modelo
        prediction = model(inputs)
        
        # Move a previsão para a CPU e converte para um array numpy
        out_pos[sample_idx, :] = prediction.detach().cpu().numpy().flatten()

        # Ajusta as previsões com base no delta_output
        out_pos[sample_idx, 0] += delta_output * (final_time / time_steps[-1])
        out_pos[sample_idx, 1] += delta_output * (final_time / time_steps[-1])
    return out_pos


In [18]:
23187/127

182.5748031496063

In [9]:

delta_input = 0.2  # time difference between input samples [s]
delta_output = 0.5  # time difference from last input to output [s]
model_1 = train_cnn(train_positions[:, :, 0, :], delta_input, delta_output)
print('Treinando modelo 2 \n')
model_2 = train_cnn(train_positions[:, :, 1, :], delta_input, delta_output)





Epoch 1/10, Loss: 5.421092167829156
Epoch 2/10, Loss: 0.02108592308545671
Epoch 3/10, Loss: 0.017169468211169284
Epoch 4/10, Loss: 0.014229267681605373
Epoch 5/10, Loss: 0.011956779434614326
Epoch 6/10, Loss: 0.01035710670814596
Epoch 7/10, Loss: 0.009612457495696817
Epoch 8/10, Loss: 0.008819554514616676
Epoch 9/10, Loss: 0.00836262059705557
Epoch 10/10, Loss: 0.007928254043348688
Epoch 1/10, Loss: 5.493330075519753
Epoch 2/10, Loss: 0.019920901618520847
Epoch 3/10, Loss: 0.016005033919209325
Epoch 4/10, Loss: 0.014795476926540384
Epoch 5/10, Loss: 0.011913913147312933
Epoch 6/10, Loss: 0.010835539775753232
Epoch 7/10, Loss: 0.010159822324818315
Epoch 8/10, Loss: 0.009327660252826116
Epoch 9/10, Loss: 0.008418735177687998
Epoch 10/10, Loss: 0.007808831440177686


Estimating input positions: 100%|██████████| 23187/23187 [00:32<00:00, 719.99it/s] 


Treinando modelo 2 



Estimating input positions: 100%|██████████| 23187/23187 [00:32<00:00, 708.60it/s]


In [ ]:
gps1_est_pos = estimate_positions(train_positions[:, :, 0, :], delta_input, delta_output,model_1)
gps2_est_pos = estimate_positions(train_positions[:, :, 1, :], delta_input, delta_output, model_2)


In [26]:
gps1_est_pos

array([[ 28.72099304, -90.89217377],
       [ 28.72099304, -90.89217377],
       [ 28.72099304, -90.89217377],
       ...,
       [ 28.69013596, -90.79010773],
       [ 28.69013214, -90.79010773],
       [ 28.69013596, -90.79010773]])

In [10]:
train_positions[:, :, 0, :].shape , gps1_est_pos.shape

((23187, 5, 2), (23187, 2))

In [27]:


lat_deltas = gps1_est_pos[:, 0] - train_positions[:, -1, 0, 0]
lon_deltas = gps1_est_pos[:, 1] - train_positions[:, -1, 0, 1]

# Compute the orientation (heading) based on the position deltas
heading = compute_ori_from_pos_delta(lat_deltas, lon_deltas)
# 2.2 - Determine relative position (converted to orientation) between vehicles
# Calculate the difference in latitude and longitude between the estimated GPS positions of vehicle 1 anp´pppd vehicle 2
lat_deltas = gps1_est_pos[:, 0] - gps2_est_pos[:, 0]
lon_deltas = gps1_est_pos[:, 1] - gps2_est_pos[:, 1]
# Compute the orientation (relative position) based on the position deltas
ori_rel = compute_ori_from_pos_delta(lat_deltas, lon_deltas)

# Compute the estimated Angle of Arrival (AOA) using the relative orientation, heading, and a reference angle
# The relative orientation is subtracted from the heading and a constant angle (pi/4) is subtracted from the result
aoa_estimation = norm_2pi(-1 * (ori_rel - heading - np.pi / 4))

# Chep

In [12]:
def predict_beam_uniformly_from_aoa(aoa):

    # Convert input to PyTorch tensor
    beam_predictions = np.zeros_like(aoa)

    beam_ori = np.arange(N_BEAMS * N_ARR) / (N_BEAMS * N_ARR - 1) * 2*np.pi - np.pi

    angl_diff_to_each_beam = aoa.reshape((-1, 1)) - beam_ori

    beam_predictions = np.argsort(abs(angl_diff_to_each_beam), axis=1)

    return beam_predictions


In [13]:

beam_pred_all = predict_beam_uniformly_from_aoa(aoa_estimation)
best_beam_pred = np.copy(beam_pred_all[:, 0]) # keep decoupled

# After analysis, we were often 1 beam short. 
pred_diff = np.array([circular_distance(a, b, sign=True)
                      for a, b in zip(best_beam_pred, y_true_beams)])

# The box sometimes is slightly rotated around it's Z axis, so we can shift our
# Beam predictions a constant offset to get better performance. Admit offsets up to 2.
# Note: this adjustment is only for the training phase
shift = -round(np.mean(pred_diff[abs(pred_diff) < 5]))
print(f'estimated_shift = {shift}')
beam_pred_all += shift
best_beam_pred += shift

# make sure the output is within 0-255
beam_pred_all[beam_pred_all>255] -= 255
beam_pred_all[beam_pred_all<0] += 255
best_beam_pred[best_beam_pred>255] -= 255
best_beam_pred[best_beam_pred<0] += 255

estimated_shift = 0


In [14]:
pred_diff_abs = np.array([circular_distance(a, b)
                          for a, b in zip(best_beam_pred, all_true_beams[:,0])])
average_beam_index_diff = np.mean(pred_diff_abs) # lower is better!

print(f'Average Beam Index Distance = {average_beam_index_diff:.2f}')

Average Beam Index Distance = 48.40


In [28]:
# "Probability of the prediction of the best beam being in the set of best k ground truth beams"
top_k = compute_acc(all_true_beams, best_beam_pred, top_k=[1, 3, 5])
print(f'Top-k = {top_k}')

# "Probability of the ground truth best beam being in the set of most likely k predicted beams"
top_k = compute_acc(beam_pred_all, all_true_beams[:, 0], top_k=[1, 3, 5])
print(f'(not used) Top-k = {top_k}')

# For practical submissions, we implement only the first way so we 
# only require the best predicted beam. 

# And finally, the score -> APL (Average Power Loss)
est_best_pwr = y_pwrs_reshaped[np.arange(n_samples), best_beam_pred]
true_best_pwr = y_pwrs_reshaped[np.arange(n_samples), all_true_beams[:, 0]]
apl = APL(true_best_pwr, est_best_pwr)
print(f'Average Power Loss = {apl:.2f} dB')

Top-k = [0.125  0.3642 0.4202]
(not used) Top-k = [0.125  0.3688 0.4357]
Average Power Loss = -10.93 dB
